<h1>Dataset Exploration</h1>

In [1]:
from pycocotools.coco import COCO
import matplotlib
import matplotlib.pyplot as plt
import os
import open3d as o3d
import cv2
import numpy as np
from PIL import Image

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
DATADIR = "cocodoom/"

dataSplit, run = "run-full-val", "run2"

annFile = '{}{}.json'.format(DATADIR,dataSplit)

In [3]:
coco = COCO(annFile)

loading annotations into memory...
Done (t=20.80s)
creating index...
index created!


In [4]:
player_positions = {"run1":[], "run2":[], "run3":[]}
motion_vectors = {"run1":[], "run2":[], "run3":[]}
USED_RUNS = ["run2"]
for run in USED_RUNS:
    with open(DATADIR+run+"/log.txt", 'r') as log_file:
        for line in log_file:
            if "player" in line:
                line = line.strip()
                tic, stats = line.split("player:")
                x, y, z, angle = stats.split(",")
    
                # Store position in the dictionary
                player_positions[run].append([float(x), float(y), float(z), float(angle)])
                if len(player_positions[run]) >= 2:
                    player_position = player_positions[run][-1]
                    prev_player_position = player_positions[run][-2]
                    
                    dx = player_position[0] - prev_player_position[0]
                    dy = player_position[1] - prev_player_position[1]
                    dz = player_position[2] - prev_player_position[2]
                    dangle = np.pi - abs(abs(player_position[3] - prev_player_position[3]) - np.pi)
                    
                    dx_relative = dx * np.cos(2 * np.pi - prev_player_position[3]) + dy * np.cos(prev_player_position[3] - 1/2 * np.pi)
                    dy_relative = dx * np.sin(2 * np.pi - prev_player_position[3]) + dy * np.sin(prev_player_position[3] - 1/2 * np.pi)
                    motion_vector = [dx_relative, dy_relative, dz, dangle]
                    motion_vectors[run].append(motion_vector)

In [5]:
def getSegmentationMask(rgb_filename):
    return rgb_filename.replace("rgb", "objects")

def getDepthMask(rgb_filename):
    return rgb_filename.replace("rgb", "depth")

In [6]:
def toPointCloud(depth):
    # https://github.com/mmatl/pyrender/issues/14#issuecomment-485881479 was used as reference
    height, width = depth.shape
    #print(depth.shape)
    fy = 200 / np.tan(1.5708 * 0.5)
    fx = 320 / np.tan(1.5708 * 0.5)

    depth = depth / 64.0
    mask = np.where(depth > 0)

    #print(depth.max(axis=1))

    x = mask[1]
    y = mask[0]

    normalized_x = (x.astype(np.float32) - width * 0.5) #/ width
    normalized_y = (y.astype(np.float32) - height * 0.5) #/ height
    
    world_x = normalized_x * depth[y, x] / fx #* 1000
    world_y = normalized_y * depth[y, x] / fy #* 1000
    world_z = depth[y, x]
    ones = np.ones(world_z.shape[0], dtype=np.float32)

    return np.vstack((world_x, world_y, world_z)).T

In [18]:
def visiblePoints(point_cloud, motion_vector):
    """
    Takes in the projected point cloud and motion vector and 
    generates a heatmap on the original image. Also produces
    a point cloud of only the points visible after movement.
    """
    angle = -1 * motion_vector[3]
    rotation_matrix = np.array([
        [np.cos(angle), 0, -1 * np.sin(angle)],[0, 1, 0],[np.sin(angle), 0, np.cos(angle)]
        ])
    #print(point_cloud.shape)
    translated_points = point_cloud - np.array([motion_vector[1], motion_vector[2], motion_vector[0]])
    #print(translated_points.shape)
    #print(rotation_matrix.shape)
    transformed_points = np.zeros(translated_points.shape)
    # This has been split up as the kernel kept dying
    for i, point in enumerate(translated_points):
        transformed_points[i] = rotation_matrix @ point

    fy = 200 / np.tan(1.5708 * 0.5)
    fx = 320 / np.tan(1.5708 * 0.5)
    x_proj = (transformed_points[:, 0] * fx / transformed_points[:, 2]) + 320 * 0.5
    y_proj = (transformed_points[:, 1] * fy / transformed_points[:, 2]) + 200 * 0.5

    #print("Made it here")

    output = np.zeros((200, 320)) # These might need to be switched
    visible_point = []
    for i, (x, y, z) in enumerate(transformed_points):
        if z > 0 and 0 <= x_proj[i] < 320 and 0 <= y_proj[i] < 200:
            output[i // 320, i % 320] = 1
            visible_point.append(transformed_points[i])

    return output, np.array(visible_point)

In [8]:
def generateHeatmap(depth, motion_vector):
    """
    Takes in the path to the depth map and the predicted or 
    actual motion vector to generate a heatmap.

    The path to the depth map should be structured as:
    runX/mapXX/depth/XXXXXX.png
    """

    point_cloud = toPointCloud(cv2.imread(DATADIR + getDepthMask(depth), cv2.IMREAD_UNCHANGED))
    heatmap, visible_point_cloud = visiblePoints(point_cloud, motion_vector)

    return heatmap, visible_point_cloud

In [9]:
imgIds = coco.getImgIds();
img = coco.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]
d = cv2.imread(DATADIR + getDepthMask(img["file_name"]), cv2.IMREAD_UNCHANGED)

p = toPointCloud(d)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(p)
o3d.io.write_point_cloud("./data.xyz", pcd)

True

In [27]:
r = cv2.imread(DATADIR + img["file_name"])
plt.imsave("rgb_frame.png", r, format="png")

In [29]:
plt.imsave("depth.png", d, format="png")

In [19]:
combined_motion_vectors = motion_vectors["run2"][10].copy()

for m in motion_vectors["run2"][11:16]:
    combined_motion_vectors[0] += m[0]
    combined_motion_vectors[1] += m[1]
    combined_motion_vectors[2] += m[2]
    combined_motion_vectors[3] += m[3]

visibility_matrix, visible_point = visiblePoints(p, combined_motion_vectors)

In [20]:
print(combined_motion_vectors)

[13.567220192888856, 18.76862818404043, -4.070399999999992, 0.29452999999999996]


In [21]:
print(visible_point.shape)
print(visibility_matrix.shape)
print(transformed_points.shape)

(39015, 3)
(200, 320)


NameError: name 'transformed_points' is not defined

In [22]:
print(visibility_matrix)

[[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [23]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(visible_point)
o3d.io.write_point_cloud("./visible_data.xyz", pcd)

True

In [24]:
cv2.imwrite('binary_image.png', visibility_matrix)

True

In [25]:
plt.imsave('binary_image.png', visibility_matrix, cmap='gray', format='png')

In [26]:
heatmap, visible_point_cloud = generateHeatmap(img["file_name"], combined_motion_vectors)
plt.imsave('binary_image.png', heatmap, cmap='gray', format='png')